# 测试流程示例

演示如何为 Xenium 与 Visium 数据计算基因程序分数、生成真值宽表，并导出解卷积输入。


## 目录
- 环境配置与基因集准备
- Xenium 基因程序打分
- Xenium 真值生成
- Visium 基因程序打分
- 数据对齐与 NPZ 导出


## 1. 环境配置与基因集设定

导入所需依赖，注册包路径，并定义用于测试的 4 组基因程序。


In [1]:
"""
预处理管线（可脚本运行）：
  - Xenium/Visium 读入、基因集打分
  - Xenium 真值计算（按 spot × broad_annotation 分组均值）
  - Visium 与细胞比例矩阵对齐，导出标准 npz（供模型使用）

注意：本版本移除了绝对路径与 os.chdir，改为 argparse 参数化；
      依赖的 compute_truth_score 当前含交互式清洗，下一步将改为无交互参数。
"""
import os
import sys
import argparse
from pathlib import Path
import numpy as np
import pandas as pd
import scanpy as sc

os.chdir('/home/vs_theg/ST_program')
# -------------------------------------------------------------------------
# 1. 添加模块路径并导入函数
# -------------------------------------------------------------------------
sys.path.append("/home/vs_theg/ST_program/CellType_GP/CellType-GP/")

from score_gene_program import score_gene_programs
from compute_truth_score import compute_truth_score

# -------------------------------------------------------------------------
# 2. 定义 4 个 Tumor Clusters 的基因集
# -------------------------------------------------------------------------
DCIS_1_genes = [
    'HPX', 'CEACAM6', 'ESR1', 'HOOK2', 'CEACAM8', 'GATA3', 'TFAP2A', 'FLNB', 'KLF5', 'CD9',
    'TPD52', 'CLDN4', 'SMS', 'DNTTIP1', 'QARS', 'C6orf132', 'KLRF1', 'LYPD3', 'SDC4', 'RHOH'
]

DCIS_2_genes = [
    'AGR3', 'S100A14', 'CEACAM8', 'KRT8', 'CCND1', 'CDH1', 'TCIM', 'AQP3', 'TACSTD2', 'LYPD3',
    'SERHL2', 'ESR1', 'CEACAM6', 'BACE2', 'DSP', 'SERPINA3', 'RORC', 'ERBB2', 'CLDN4', 'DMKN'
]

Prolif_Invasive_genes = [
    'CENPF', 'MKI67', 'TOP2A', 'PCLAF', 'STC1', 'RTKN2', 'TUBA4A', 'MDM2', 'HMGA1', 'C2orf42',
    'POLR2J3', 'PTRHD1', 'SRPK1', 'EIF4EBP1', 'SQLE', 'SH3YL1', 'THAP2', 'NPM3', 'LAG3', 'FOXA1'
]

Invasive_Tumor_genes = [
    'ABCC11', 'SERHL2', 'TCIM', 'FASN', 'AR', 'PTRHD1', 'TRAF4', 'USP53', 'SCD', 'SQLE',
    'MYO5B', 'DNAAF1', 'FOXA1', 'EPCAM', 'CTTN', 'MLPH', 'ELF3', 'ANKRD30A', 'ENAH', 'KARS'
]

# 合并所有基因集用于评分
gene_sets_to_score = {
    'DCIS_1_score': DCIS_1_genes,
    'DCIS_2_score': DCIS_2_genes,
    'Prolif_Invasive_score': Prolif_Invasive_genes,
    'Invasive_Tumor_score': Invasive_Tumor_genes,
}
def main():
    """
    主函数：设置参数、检查文件，并返回配置。
    """
    parser = argparse.ArgumentParser(description="Preprocess ST data: scoring + truth + aligned NPZ")
    
    # --- 1. 定义参数 ---
    # 使用 pathlib.Path 作为类型，使其成为可调用的路径构造函数
    # 默认路径是相对于项目根目录的相对路径
    parser.add_argument("--xenium", type=Path, default=Path("CellType_GP/DATA/xdata.h5"), 
                        help="Xenium AnnData (.h5ad/.h5) path")
    parser.add_argument("--visium", type=Path, default=Path("CellType_GP/DATA/vdata.h5"), 
                        help="Visium AnnData (.h5ad/.h5) path")
    parser.add_argument("--fractions", type=Path, default=Path("CellType_GP/DATA/spot_cluster_fraction_matrix.csv"), 
                        help="Spot-celltype fractions CSV (index=spot)")
    parser.add_argument("--x2v-map", type=Path, default=Path("CellType_GP/DATA/xenium_to_visium_transcript_mapping.csv"), 
                        help="Xenium->Visium barcode mapping CSV")
    parser.add_argument("--outdir", type=Path, default=Path("CellType_GP/DATA/preprocessing_outputs"), 
                        help="Output directory for processed files")

    # --- 2. 解析参数 ---
    # 在 Notebook 中使用 parse_args([]) 是一个很好的实践，
    # 它强制 argparse 使用在 add_argument 中定义的 `default` 值，
    # 而不是去读取（不存在的）命令行输入。
    args = parser.parse_args([]) 
    
    # --- 3. 创建输出目录 ---
    outdir = args.outdir
    outdir.mkdir(parents=True, exist_ok=True)
    
    print(f"Output directory set to: {outdir.absolute()}")

    # --- 4. 检查关键文件是否存在 ---
    print("Checking file paths...")
    key_files_to_check = {
        "xenium": args.xenium,
        "visium": args.visium,
        "fractions": args.fractions,
        "x2v-map": args.x2v_map
    }

    for name, path in key_files_to_check.items():
        exists = path.exists()
        print(f"  - {name}: {path} ... {'✅ Exists' if exists else '❌ NOT FOUND'}")
        
        # 如果关键文件（如此处的 xenium）缺失，则应立即报错
        if not exists and name == "xenium": 
            raise FileNotFoundError(
                f"Key file '{name}' missing from expected path: {path.absolute()}\n"
                "Please ensure your data is correctly placed in CellType_GP/DATA/"
            )

    # --- 5. 返回配置 ---
    # （原始代码中此处的重复代码块已被移除）
    
    print(f"Configuration loaded successfully. Using Xenium data from: {args.xenium.absolute()}")
    
    # 返回 args 和 outdir，使其在 Notebook 的全局作用域中可用
    return args, outdir


/home/vs_theg/anaconda3/envs/DL/lib/python3.9/site-packages/xarray_schema/__init__.py:1: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import DistributionNotFound, get_distribution
/home/vs_theg/anaconda3/envs/DL/lib/python3.9/site-packages/numba/core/decorators.py:246: RuntimeWarning: nopython is set for njit and is ignored
  warnings.warn('nopython is set for njit and is ignored', RuntimeWarning)


In [2]:
print("Running setup...")
# 调用 main() 并捕获返回值，将其存为全局变量
try:
    args, outdir = main()
except FileNotFoundError as e:
    print(e)
    # 如果文件未找到，你可能希望在这里处理错误，而不是让后续单元格失败
except Exception as e:
    print(f"An unexpected error occurred during setup: {e}")
    
print("Setup complete.")

Running setup...
Output directory set to: /home/vs_theg/ST_program/CellType_GP/DATA/preprocessing_outputs
Checking file paths...
  - xenium: CellType_GP/DATA/xdata.h5 ... ✅ Exists
  - visium: CellType_GP/DATA/vdata.h5 ... ✅ Exists
  - fractions: CellType_GP/DATA/spot_cluster_fraction_matrix.csv ... ✅ Exists
  - x2v-map: CellType_GP/DATA/xenium_to_visium_transcript_mapping.csv ... ✅ Exists
Configuration loaded successfully. Using Xenium data from: /home/vs_theg/ST_program/CellType_GP/DATA/xdata.h5
Setup complete.


## 2. Xenium 基因程序打分

在 Xenium 单细胞矩阵上运行 `score_gene_programs`，并把结果落盘。


In [3]:
# ---------------------------------------------------------------------
# 3. Xenium 数据打分
# ---------------------------------------------------------------------
adata_x = sc.read(str(args.xenium))
score_gene_programs(adata_x, gene_sets_to_score, platform="xenium", output_dir=str(outdir / "xenium_scores"))
adata_x.write(str(outdir / "xdata_processed.h5"))

🔹 正在处理: DCIS_1_score
✅ 完成: DCIS_1_score (20 基因)

🔹 正在处理: DCIS_2_score
✅ 完成: DCIS_2_score (20 基因)

🔹 正在处理: Prolif_Invasive_score
✅ 完成: Prolif_Invasive_score (20 基因)

🔹 正在处理: Invasive_Tumor_score
✅ 完成: Invasive_Tumor_score (20 基因)


🎉 所有基因集打分完成（使用平均表达），结果已保存至 CellType_GP/DATA/preprocessing_outputs/xenium_scores/



## 3. 生成 Xenium 真值宽表

合并 Xenium 注释后计算聚合得分，输出到 `truth_output/` 目录。


In [4]:
# ---------------------------------------------------------------------
# 4. Xenium 真值计算（合并条形码映射；清洗后按 group 平均）
# ---------------------------------------------------------------------
x2v = pd.read_csv(args.x2v_map)
# 自动检测 Xenium 细胞 ID 列 / auto-detect xenium cell id column
possible_keys = ["cell_id", "Barcode", "cell_ID", "xenium_cell_id"]
left_key = next((k for k in possible_keys if k in adata_x.obs.columns), None)
if left_key is None:
    raise ValueError("❌ 在 adata_x.obs 中找不到匹配的 cell id 列（尝试列：" + ", ".join(possible_keys) + ")")

# 合并到 obs 上（左连接），保留 Visium 条形码 / merge mapping into obs
adata_x.obs = adata_x.obs.merge(
    x2v[["xenium_cell_id", "transcript_level_visium_barcode"]],
    how="left",
    left_on=left_key,
    right_on="xenium_cell_id"
)
if "xenium_cell_id" in adata_x.obs.columns:
    adata_x.obs.drop(columns=["xenium_cell_id"], inplace=True)

before = adata_x.n_obs
adata_x = adata_x[adata_x.obs["transcript_level_visium_barcode"].notna()].copy()
after = adata_x.n_obs
print(f"✅ 去除无 transcript_level_visium_barcode 的细胞：删除 {before - after}，剩余 {after}")

# 计算真值（注意：当前 compute_truth_score 含交互式清洗，下一步将去交互化）
compute_truth_score(
    adata_x,
    output_dir=str(outdir / "truth_output"),
    spot_col='transcript_level_visium_barcode',
    celltype_col='broad_annotation',
    drop_columns=[]
)


/home/vs_theg/anaconda3/envs/DL/lib/python3.9/site-packages/anndata/_core/aligned_df.py:68: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/home/vs_theg/ST_program/CellType_GP/CellType-GP/compute_truth_score.py:69: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby([spot_col, celltype_col])[score_cols]


✅ 去除无 transcript_level_visium_barcode 的细胞：删除 22938，剩余 135092
🚀 开始计算 Xenium truth score, 输出路径: CellType_GP/DATA/preprocessing_outputs/truth_output

🧾 当前 adata.obs 列如下（可直接复制粘贴）：

cell_id, transcript_counts, control_probe_counts, control_codeword_counts, total_counts, cell_area, nucleus_area, region, Barcode, Cluster, broad_annotation, Stromal_score, Prolif_Invasive_Tumor_score, Perivascular_Like_score, Myoepi_KRT15_score, Myoepi_ACTA2_score, Mast_Cells_score, Macrophages_2_score, Macrophages_1_score, LAMP3_DCs_score, IRF7_DCs_score, Invasive_Tumor_score, Endothelial_score, DCIS_2_score, DCIS_1_score, CD8_T_Cells_score, CD4_T_Cells_score, B_Cells_score, DCIS_1_score_norm, DCIS_2_score_norm, Prolif_Invasive_score_norm, Invasive_Tumor_score_norm, transcript_level_visium_barcode

💡 提示：你可以复制上面这一行，然后粘贴要删除的列（或部分列）

✅ 未删除任何列
✨ 清洗完成：保留 33 列。

🧩 检测到 4 个得分列：['DCIS_1_score_norm', 'DCIS_2_score_norm', 'Prolif_Invasive_score_norm', 'Invasive_Tumor_score_norm'] ...


/home/vs_theg/ST_program/CellType_GP/CellType-GP/compute_truth_score.py:75: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby([spot_col, celltype_col])
/home/vs_theg/ST_program/CellType_GP/CellType-GP/compute_truth_score.py:91: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  truth_wide = truth_result.pivot_table(index=spot_col, columns=celltype_col, values=program_cols)


✅ 分组平均完成: 27671 行
✅ 宽表完成: 3953 × 29
💾 保存结果：
  ├─ 细胞级 truth_score：CellType_GP/DATA/preprocessing_outputs/truth_output/truth_score.csv
  ├─ 分组均值 truth_result：CellType_GP/DATA/preprocessing_outputs/truth_output/truth_result_grouped.csv
  └─ 宽表 truth_result(wide)：CellType_GP/DATA/preprocessing_outputs/truth_output/truth_result(wide).csv

🎉 Xenium truth score 计算完成！


,spot,B_Cells+DCIS_1_score_norm,DCIS+DCIS_1_score_norm,Endothelial+DCIS_1_score_norm,Invasive_Tumor+DCIS_1_score_norm,Myeloid+DCIS_1_score_norm,Stromal+DCIS_1_score_norm,T_cells+DCIS_1_score_norm,B_Cells+DCIS_2_score_norm,DCIS+DCIS_2_score_norm,...,Myeloid+Invasive_Tumor_score_norm,Stromal+Invasive_Tumor_score_norm,T_cells+Invasive_Tumor_score_norm,B_Cells+Prolif_Invasive_score_norm,DCIS+Prolif_Invasive_score_norm,Endothelial+Prolif_Invasive_score_norm,Invasive_Tumor+Prolif_Invasive_score_norm,Myeloid+Prolif_Invasive_score_norm,Stromal+Prolif_Invasive_score_norm,T_cells+Prolif_Invasive_score_norm
0,AACACGTGCATCGCAC-1,0.375284,NaN,NaN,NaN,0.000000,0.220213,NaN,0.147681,NaN,...,0.056702,0.220103,NaN,0.404542,NaN,NaN,NaN,0.000000,0.217262,NaN
1,AACACTTGGCAAGGAA-1,NaN,0.575892,0.277421,NaN,0.108926,0.146012,0.116139,NaN,0.845507,...,0.094999,0.161601,0.097661,NaN,0.310731,0.177289,NaN,0.184925,0.196339,0.217015
2,AACAGGAAGAGCATAG-1,NaN,NaN,NaN,NaN,NaN,0.321083,NaN,NaN,NaN,...,NaN,0.344991,NaN,NaN,NaN,NaN,NaN,NaN,0.231498,NaN
3,AACAGGATTCATAGTT-1,NaN,NaN,0.247145,NaN,0.062199,0.276357,0.171217,NaN,NaN,...,0.151933,0.390091,0.205113,NaN,NaN,0.275294,NaN,0.212327,0.322324,0.331868
4,AACAGGTTATTGCACC-1,NaN,NaN,NaN,NaN,0.076516,0.134703,NaN,NaN,NaN,...,0.162328,0.183649,NaN,NaN,NaN,NaN,NaN,0.154542,0.201818,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3948,TGTTGGAACCTTCCGC-1,NaN,NaN,NaN,NaN,NaN,0.197189,NaN,NaN,NaN,...,NaN,0.303076,NaN,NaN,NaN,NaN,NaN,NaN,0.321613,NaN
3949,TGTTGGAACGAGGTCA-1,NaN,NaN,NaN,NaN,0.128381,0.203931,NaN,NaN,NaN,...,0.104531,0.169283,NaN,NaN,NaN,NaN,NaN,0.141015,0.187936,NaN
3950,TGTTGGAAGCTCGGTA-1,0.160356,0.556254,0.238386,NaN,0.105313,0.131702,0.156128,0.104129,0.735014,...,0.089908,0.151088,0.067906,0.180778,0.331768,0.193754,NaN,0.212019,0.204104,0.212550
3951,TGTTGGATGGACTTCT-1,NaN,NaN,0.157251,0.473101,0.089738,0.169298,0.290752,NaN,NaN,...,0.137208,0.295425,0.324691,NaN,NaN,0.114060,0.524094,0.216858,0.213031,0.333305


## 4. Visium 基因程序打分

对 Visium 空间数据重复基因程序打分，并保存处理后的 AnnData 文件。


In [5]:
# ---------------------------------------------------------------------
# 5. Visium 数据打分
# ---------------------------------------------------------------------
adata_v = sc.read(str(args.visium))
score_gene_programs(adata_v, gene_sets_to_score, platform="visium", output_dir=str(outdir / "visium_scores"))
adata_v.write(str(outdir / "vdata_processed.h5"))


🔹 正在处理: DCIS_1_score
✅ 完成: DCIS_1_score (20 基因)

🔹 正在处理: DCIS_2_score
✅ 完成: DCIS_2_score (20 基因)

🔹 正在处理: Prolif_Invasive_score
⚠️ 缺失基因 (Prolif_Invasive_score): POLR2J3
✅ 完成: Prolif_Invasive_score (19 基因)

🔹 正在处理: Invasive_Tumor_score
✅ 完成: Invasive_Tumor_score (20 基因)


🎉 所有基因集打分完成（使用平均表达），结果已保存至 CellType_GP/DATA/preprocessing_outputs/visium_scores/



## 5. 数据对齐与 NPZ 导出

同步 Visium 得分、空间坐标与细胞类型比例矩阵，最终生成 `spot_data_full.npz`。


In [14]:
# ---------------------------------------------------------------------
# 6. 对齐 + 生成标准化 NPZ 文件
#    重新计算细胞比例（不再使用外部 fractions 文件），按 Xenium 的 broad_annotation 聚合
# ---------------------------------------------------------------------
# 使用 Xenium obs 中的条形码与大类注释，统计每个 Visium spot 的各大类细胞占比
x_obs = adata_x.obs[["transcript_level_visium_barcode", "broad_annotation"]].dropna(subset=["transcript_level_visium_barcode", "broad_annotation"])
spot_cluster_counts = (
    x_obs.groupby(["transcript_level_visium_barcode", "broad_annotation"])
    .size()
    .reset_index(name="cell_count")
)
spot_total_counts = (
    spot_cluster_counts.groupby("transcript_level_visium_barcode")["cell_count"]
    .sum()
    .reset_index(name="total_cells")
)
spot_cluster_stats = spot_cluster_counts.merge(spot_total_counts, on="transcript_level_visium_barcode")
spot_cluster_stats["fraction"] = spot_cluster_stats["cell_count"] / spot_cluster_stats["total_cells"]
fractions = spot_cluster_stats.pivot_table(
    index="transcript_level_visium_barcode",
    columns="broad_annotation",
    values="fraction",
    fill_value=0
)
fractions.index = fractions.index.astype(str)

# 对齐顺序（严格使用交集，避免 KeyError）/ strict intersection
target_spots = pd.Index(fractions.index.astype(str))
vis_spots = adata_v.obs_names.astype(str)
common = target_spots.intersection(vis_spots)
missing_in_visium = target_spots.difference(vis_spots)
missing_in_fraction = vis_spots.difference(target_spots)
if len(missing_in_visium):
    print(f"⚠️ 在 Visium 中缺失 {len(missing_in_visium)} 个 fractions 索引（已丢弃）：前3个示例：{list(missing_in_visium[:3])}")
if len(missing_in_fraction):
    print(f"⚠️ 在 fractions 中缺失 {len(missing_in_fraction)} 个 Visium 索引（已丢弃）：前3个示例：{list(missing_in_fraction[:3])}")

# 重排为相同顺序 / reorder both
fractions = fractions.loc[common]
adata_v = adata_v[common].copy()
print(f"✅ Visium 与 fractions 已对齐，共 {adata_v.n_obs} 个公共 spot。")

# 生成矩阵 / build matrices
# 程序列按定义顺序提取，保证与 gene_sets_to_score 对应；过滤不存在列
# desired_cols = [f"{k}_norm" for k in gene_sets_to_score.keys()] # 如果是未标准化，则注释掉
# score_cols = [c for c in desired_cols if c in adata_v.obs.columns]

score_cols = [c for c in gene_sets_to_score if c in adata_v.obs.columns]

if len(score_cols) == 0:
    raise RuntimeError("❌ 未在 Visium obs 中找到任何 *_score_norm 列，请先成功打分。")

visium_score = adata_v.obs[score_cols].values.T            # (P, S)
coords = adata_v.obsm["spatial"] if "spatial" in adata_v.obsm_keys() else np.zeros((adata_v.n_obs, 2))

spot_names = fractions.index.values.astype(str)
# 细胞类型名称从 fractions 列派生，确保顺序一致 / derive ct names from fractions columns
celltype_names = np.array(fractions.columns.values, dtype=str)
program_names = np.array(score_cols, dtype=str)

# 保存坐标 CSV（便于可视化）
coords_df = pd.DataFrame(coords, columns=["x", "y"])
coords_df.to_csv(outdir / "vdata_spatial_coords.csv", index=False)
print("✅ 已保存到", outdir / "vdata_spatial_coords.csv", "形状：", coords_df.shape)

# 保存 npz（标准输入包）
npz_path = outdir / "spot_data_full.npz"
np.savez_compressed(
    npz_path,
    spot_cluster_fraction_matrix=fractions.values.astype(np.float32),
    coords=coords.astype(np.float32),
    visium_score=visium_score.astype(np.float32),
    spot_names=spot_names,
    celltype_names=celltype_names,
    program_names=program_names,
)
print("🎉 成功保存：", npz_path)

print("✅ 矩阵形状：")
print("  spot_cluster_fraction_matrix:", fractions.shape)
print("  visium_score:", visium_score.shape)
print("  coords:", coords.shape)


/tmp/ipykernel_3927106/1763122239.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  x_obs.groupby(["transcript_level_visium_barcode", "broad_annotation"])
/tmp/ipykernel_3927106/1763122239.py:19: FutureWarning: The default value of observed=False is deprecated and will change to observed=True in a future version of pandas. Specify observed=False to silence this warning and retain the current behavior
  fractions = spot_cluster_stats.pivot_table(


✅ Visium 与 fractions 已对齐，共 3953 个公共 spot。
✅ 已保存到 CellType_GP/DATA/preprocessing_outputs/vdata_spatial_coords.csv 形状： (3953, 2)
🎉 成功保存： CellType_GP/DATA/preprocessing_outputs/spot_data_full.npz
✅ 矩阵形状：
  spot_cluster_fraction_matrix: (3953, 6)
  visium_score: (13, 3953)
  coords: (3953, 2)


In [17]:
fractions.to_csv('/home/vs_theg/ST_program/CellType_GP/DATA/spot_cluster_fraction_matrix.csv`')

## 6. 输出文件的地址

- `/home/vs_theg/ST_program/CellType_GP/DATA/xenium_scores/`
- `/home/vs_theg/ST_program/CellType_GP/DATA/xdata_processed.h5`

### 真值文件
- `/home/vs_theg/ST_program/CellType_GP/DATA/truth_output/` 
包含：
  - `truth_result(wide).csv`
  - `truth_score.csv`
  - `truth_result_grouped.csv`  

- `/home/vs_theg/ST_program/CellType_GP/DATA/visium_scores/`
- `/home/vs_theg/ST_program/CellType_GP/DATA/vdata_processed.h5`

### 训练输入文件
- `/home/vs_theg/ST_program/CellType_GP/DATA/vdata_spatial_coords.csv`
- `/home/vs_theg/ST_program/CellType_GP/DATA/visium_program_scores.csv`
- `/home/vs_theg/ST_program/CellType_GP/DATA/spot_cluster_fraction_matrix.csv`

- `/home/vs_theg/ST_program/CellType_GP/DATA/spot_data_full.npz`


!ls /home/vs_theg/ST_program/CellType_GP/DATA/preprocessing_outputs

**spot_data_full.npz**
vdata_processed.h5	      visium_scores	  xenium_scores
truth_output	    vdata_spatial_coords.csv  xdata_processed.h5
